In [1]:
import pandas as pd 
import akshare as ak
import os 
import fitz
import math

/Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/akshare/__init__.py:2606: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.1.0 及以上版本！
  warnings.warn(


In [15]:
# values_to_remove = ["华为2022.pdf", "万科2022.pdf", "601966.SH-玲珑轮胎-玲珑轮胎 山东玲珑轮胎股份有限公司2022年可持续发展报告英文版-2023-04-29.pdf", "002129.SZ-TCL中环-TCL中环 TCL中环新能源科技股份有限公司2022年度可持续发展报告(英文版)-2023-03-29.pdf", "000726.SZ-鲁泰A-鲁泰Ａ 鲁泰纺织可持续发展报告2022-英文版-2023-04-12.pdf"]
# df_img = df_img[~df_img['PDF_name'].isin(values_to_remove)]

# 获取df_PDF

In [33]:
# 获取PDF页数
def pages(PDF_path):
    pdf_file = fitz.open(PDF_path)
    return pdf_file.page_count

In [37]:
# 页面对角线距离
def diagonal(PDF_path):
    pdf_file = fitz.open(PDF_path)
    width, height = pdf_file[3].rect.width, pdf_file[2].rect.height
    
    return math.sqrt(width**2 + height**2)

In [40]:
# 获取上市公司代码
def corp_code(PDF_name):
    return str(PDF_name.split("-")[0][0:6]) if PDF_name.split("-")[0][-2:] != 'HK' else str(PDF_name.split("-")[0][0:5])

def corp_market(PDF_name):
    return PDF_name.split("-")[0][-2:]

In [47]:
def get_df_PDF(filepath, PDF_folder, PDF_year, PDF_type):
    # 读入df_img，保存着距离信息
    df_img = pd.read_excel(filepath)
    
    if "Unnamed: 0" in df_img.columns:
        df_img = df_img.drop("Unnamed: 0", axis=1)
        
    values_to_remove = ["华为2022.pdf", "万科2022.pdf", "601966.SH-玲珑轮胎-玲珑轮胎 山东玲珑轮胎股份有限公司2022年可持续发展报告英文版-2023-04-29.pdf", "002129.SZ-TCL中环-TCL中环 TCL中环新能源科技股份有限公司2022年度可持续发展报告(英文版)-2023-03-29.pdf", "000726.SZ-鲁泰A-鲁泰Ａ 鲁泰纺织可持续发展报告2022-英文版-2023-04-12.pdf"]
    df_img = df_img[~df_img['PDF_name'].isin(values_to_remove)]
        
    # 准备df_PDF
    df_PDF = pd.DataFrame()
    df_PDF['PDF_name'] = df_img['PDF_name'].unique()
    df_PDF['Corp_code'] = df_PDF['PDF_name'].apply(corp_code)
    df_PDF['Corp_market'] = df_PDF['PDF_name'].apply(corp_market)
    df_PDF['Pub_year'] = PDF_year
    df_PDF['Pub_type'] = PDF_type
    
    # 获取PDF页数
    df_PDF['PDF_pages'] = df_PDF['PDF_name'].apply(lambda x: pages(os.path.join(PDF_folder, x)))
    
    # 获取PDF内图片数量
    df_PDF = df_PDF.merge(df_img.groupby('PDF_name')['page'].count().reset_index(name= 'img_num'),
                     on='PDF_name', how='left')
    
    # 获取PDF内图片大小（均值）
    df_img['img_size'] = df_img.apply(lambda row: abs(row["x1"] - row["x0"]) * abs(row["y1"] - row["y0"]),
                                      axis=1)
    df_PDF = df_PDF.merge(df_img.groupby('PDF_name')['img_size'].mean().reset_index(name= 'img_size_avg'),
                     how='left', on='PDF_name')
    
    # 计算图片数量与页数比
    df_PDF['img_num_pages_ratio'] = df_PDF.apply(lambda row: row['img_num'] / row['PDF_pages'], axis=1)

    # 计算图片大小和页数比
    df_PDF['img_size_pages_ratio'] = df_PDF.apply(lambda row: row['img_size_avg'] / row['PDF_pages'], axis=1)
    
    # 计算 图文空间距离(平均）与 页面对角线距离 比
    diagonal_series = df_PDF['PDF_name'].apply(lambda x: diagonal(os.path.join(PDF_folder, x)))
    diagonal_series.index = df_PDF['PDF_name']
    
    diagonal_compute = pd.merge(diagonal_series.reset_index(name = 'diag'), 
                           df_img.groupby('PDF_name')['distance'].mean().reset_index(name = 'dist_avg'),
                           on='PDF_name', how = 'left')
    diagonal_compute['dist_diag_ratio'] = diagonal_compute.apply(lambda row: row['dist_avg'] / row['diag'], 
                                                                 axis=1)
    df_PDF = pd.merge(df_PDF, diagonal_compute[['PDF_name', 'dist_diag_ratio']],
                      on='PDF_name', how = 'left')
    
    # 保存文件
    df_PDF.to_excel("./Playground/04_DataIntegration/00_separate/df_PDF_{}_{}.xlsx".format(PDF_type, PDF_year))

    return df_PDF

In [53]:
get_df_PDF("./Playground/03_TextImgMatch/distance/ESG_2022_distance.xlsx",
          "./Playground/00_DataBase/01_ESGReport_PDF/2022",
          2022, "ESG")

,PDF_name,Corp_code,Corp_market,Pub_year,Pub_type,PDF_pages,img_num,img_size_avg,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio
0,300011.SZ-鼎汉技术-鼎汉技术 2022年社会责任暨环境、社会及管治(ESG)报告-...,300011,SZ,2022,ESG,47,118,234082.815567,2.510638,4980.485438,0.347275
1,300021.SZ-大禹节水-大禹节水 2022年度环境、社会及公司治理(ESG)报告-20...,300021,SZ,2022,ESG,38,219,50995.114252,5.763158,1341.976691,0.377827
2,000553.SZ-安道麦A-安道麦A 2022年度环境、社会及公司治理报告-2023-07...,000553,SZ,2022,ESG,143,145,493534.342545,1.013986,3451.289109,0.185911
3,600779.SH-水井坊-水井坊 水井坊2022年度环境、社会、公司治理(ESG)报告-2...,600779,SH,2022,ESG,61,87,243267.877227,1.426230,3987.997987,0.335838
4,601600.SH-中国铝业-中国铝业 中国铝业2022年度社会责任暨环境、社会与管治报告-...,601600,SH,2022,ESG,92,88,188413.610568,0.956522,2047.974028,0.237009
...,...,...,...,...,...,...,...,...,...,...,...
593,002506.SZ-协鑫集成-协鑫集成 协鑫集成科技股份有限公司2022年度环境、社会及管治...,002506,SZ,2022,ESG,44,100,124995.912834,2.272727,2840.816201,0.297456
594,600487.SH-亨通光电-亨通光电 亨通光电2022年环境、社会及治理报告-2023-0...,600487,SH,2022,ESG,39,74,409709.302261,1.897436,10505.366725,0.362397
595,600874.SH-创业环保-创业环保 创业环保2022年环境、社会及管治报告-2023-0...,600874,SH,2022,ESG,43,3,353974.728585,0.069767,8231.970432,0.159869
596,601918.SH-新集能源-新集能源 新集能源2022年环境、社会和治理报告-2023-0...,601918,SH,2022,ESG,76,76,481020.231220,1.000000,6329.213569,NaN


# 合并df_PDF

In [58]:
df_PDF_SUS_2020 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_SUS_2020.xlsx")
df_PDF_SUS_2021 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_SUS_2021.xlsx")
df_PDF_SUS_2022 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_SUS_2022.xlsx")
df_PDF_ESG_2020 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_ESG_2020.xlsx")
df_PDF_ESG_2021 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_ESG_2021.xlsx")
df_PDF_ESG_2022 = pd.read_excel("./Playground/04_DataIntegration/00_separate/df_PDF_ESG_2022.xlsx")

df_PDF_main = pd.concat([df_PDF_ESG_2020, df_PDF_ESG_2021, df_PDF_ESG_2022, 
                         df_PDF_SUS_2020, df_PDF_SUS_2021, df_PDF_SUS_2022])
df_PDF_main = df_PDF_main.drop('Unnamed: 0', axis=1)

# 保存df
df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main.xlsx",
                    index=False)

# 读取PDF

In [37]:
df_PDF_main = pd.read_excel("./Playground/04_DataIntegration/df_PDF_main.xlsx",
                           dtype={'Corp_code': str})
df_PDF_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PDF_name              1046 non-null   object 
 1   Corp_code             1046 non-null   object 
 2   Corp_market           1046 non-null   object 
 3   Pub_year              1046 non-null   int64  
 4   Pub_type              1046 non-null   object 
 5   PDF_pages             1046 non-null   int64  
 6   img_num               1046 non-null   int64  
 7   img_size_avg          1046 non-null   float64
 8   img_num_pages_ratio   1046 non-null   float64
 9   img_size_pages_ratio  1046 non-null   float64
 10  dist_diag_ratio       961 non-null    float64
dtypes: float64(4), int64(3), object(4)
memory usage: 90.0+ KB


In [38]:
# 补全股票代码
df_PDF_main['Corp_code'] = df_PDF_main.Corp_code.apply(lambda x: f'{"0"*(6-len(x))}{x}' if len(x)<6 else x)

In [39]:
# 创建上市公司名字
df_PDF_main['Corp_name'] = df_PDF_main['PDF_name'].apply(lambda x: x.split('-')[1])

# 企业行业类型导入

In [40]:
stock_industry_category_cninfo_df = pd.read_excel("./Playground/00_DataBase/02_Stock/证监会行业分类标准.xlsx")

if 'Unnamed: 0' in stock_industry_category_cninfo_df.columns:
    stock_industry_category_cninfo_df = stock_industry_category_cninfo_df.drop('Unnamed: 0', axis=1)
    
stock_industry_category_cninfo_df

,类目编码,类目名称,终止日期,行业类型,行业类型编码,类目名称英文,父类编码,分级
0,0,证监会行业分类,NaN,证监会行业分类标准,8001,CSRC Industry Classification,008,0
1,A,农、林、牧、渔业,NaN,证监会行业分类标准,8001,Agriculture,0,0
2,A01,农业,NaN,证监会行业分类标准,8001,Agriculture,A,1
3,A02,林业,NaN,证监会行业分类标准,8001,Forestry,A,1
4,A03,畜牧业,NaN,证监会行业分类标准,8001,Animal Husbandry,A,1
...,...,...,...,...,...,...,...,...
105,R87,文化艺术业,NaN,证监会行业分类标准,8001,Culture and Art,R,1
106,R88,体育,NaN,证监会行业分类标准,8001,Sports,R,1
107,R89,娱乐业,NaN,证监会行业分类标准,8001,Entertainment,R,1
108,S,综合,NaN,证监会行业分类标准,8001,Conglomerates,0,0


In [41]:
def industry_info(corp_code, stock_industry_category_cninfo_df=stock_industry_category_cninfo_df):
    if len(str(corp_code)) == 6:
        industry_name_list = ak.stock_profile_cninfo(symbol=str(corp_code))['所属行业'].values.tolist()
        if len(industry_name_list) != 0:
            industry_name = industry_name_list[0]
            industry_code = stock_industry_category_cninfo_df[stock_industry_category_cninfo_df['类目名称'] == industry_name]['类目编码'].values.tolist()[-1]
        
            return industry_code
        else:
            print(f'ZeroInfoError: corp_code {corp_code} has no info.')
            return None
    else:
        print(f'Unregularized corp_code found: {corp_code}')
        return None

In [42]:
df_PDF_main['Corp_industry_code'] = df_PDF_main['Corp_code'].apply(industry_info)

ZeroInfoError: corp_code 603157 has no info.
ZeroInfoError: corp_code 101551 has no info.
ZeroInfoError: corp_code 001138 has no info.
ZeroInfoError: corp_code 000941 has no info.
ZeroInfoError: corp_code 003347 has no info.
ZeroInfoError: corp_code 006198 has no info.


## 虚拟变量：污染行业和垄断行业

In [43]:
def polluting_industry(corp_industry_code, poll_list = ['C25', 'C26', 'C28', 'C31', 'C32', 'C33']):
    """
    Return True if corp in polluting industry else False.
    """
    return corp_industry_code in poll_list

def monopoly_industry(corp_industry_code, mono_list = ['D44', 'D45', 'D46']):
    """
    Return True if corp in monopolistic industry else False.
    """
    return corp_industry_code in mono_list

In [44]:
df_PDF_main['Mono_idst'] = df_PDF_main['Corp_industry_code'].apply(monopoly_industry)
df_PDF_main['Poll_idst'] = df_PDF_main['Corp_industry_code'].apply(polluting_industry)

In [46]:
# df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main2.xlsx")

In [48]:
df_PDF_main = df_PDF_main[['PDF_name', 'Corp_code', 'Corp_name', 'Corp_market', 'Pub_year', 'Pub_type',
       'PDF_pages', 'img_num', 'img_size_avg', 'img_num_pages_ratio',
       'img_size_pages_ratio', 'dist_diag_ratio', 
       'Corp_industry_code', 'Mono_idst', 'Poll_idst']]

In [49]:
df_PDF_main

,PDF_name,Corp_code,Corp_name,Corp_market,Pub_year,Pub_type,PDF_pages,img_num,img_size_avg,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio,Corp_industry_code,Mono_idst,Poll_idst
0,601066.SH-中信建投-2020年度社会责任暨环境、社会与管治报告-2021-04-0...,601066,中信建投,SH,2020,ESG,76,9,42387.210437,0.118421,557.726453,0.350460,J67,False,False
1,600803.SH-新奥股份-新奥天然气股份有限公司2020年度环境、社会及管治报告(中文版...,600803,新奥股份,SH,2020,ESG,44,153,128895.980539,3.477273,2929.454103,0.333972,D45,True,False
2,600600.SH-青岛啤酒-环境、社会及管治报告-2021-03-29.pdf,600600,青岛啤酒,SH,2020,ESG,32,99,125883.408804,3.093750,3933.856525,0.340635,C15,False,False
3,300759.SZ-康龙化成-康龙化成 2020年度社会责任暨环境、社会及管治报告-2021...,300759,康龙化成,SZ,2020,ESG,28,25,27196.530652,0.892857,971.304666,0.273792,M73,False,False
4,002242.SZ-九阳股份-九阳股份 2020年环境、社会及管治报告-2021-06-11...,002242,九阳股份,SZ,2020,ESG,51,52,61956.921173,1.019608,1214.841592,0.212756,C38,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,600201.SH-生物股份-生物股份 金宇生物技术股份有限公司2022年年度可持续发展报告...,600201,生物股份,SH,2022,SUS,36,35,102463.497340,0.972222,2846.208259,0.319214,C27,False,False
1042,600588.SH-用友网络-用友网络 用友网络2022年度可持续发展(ESG)报告-202...,600588,用友网络,SH,2022,SUS,33,31,109075.470388,0.939394,3305.317284,0.252490,I65,False,False
1043,600989.SH-宝丰能源-宝丰能源 宁夏宝丰能源集团股份有限公司2022年可持续发展报告...,600989,宝丰能源,SH,2022,SUS,35,58,199669.598451,1.657143,5704.845670,0.291486,C26,False,True
1044,600019.SH-宝钢股份-宝钢股份 宝山钢铁股份有限公司2022年度可持续发展报告-20...,600019,宝钢股份,SH,2022,SUS,73,227,157117.564714,3.109589,2152.295407,0.292533,C31,False,True


# 财务数据
ROA，ROE，成立时长，总资产，专利数
## 专利数数据来源
https://bbs.pinggu.org/thread-11541330-1-1.html

In [66]:
BalanceSheet_2020 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股资产负债表2020.xlsx",
                                 dtype={'股票代码': str})
BalanceSheet_2020['所属年份'] = '2020'
BalanceSheet_2021 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股资产负债表2021.xlsx",
                                 dtype={'股票代码': str})
BalanceSheet_2021['所属年份'] = '2021'
BalanceSheet_2022 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股资产负债表2022.xlsx",
                                 dtype={'股票代码': str})
BalanceSheet_2022['所属年份'] = '2022'
BalanceSheet = pd.concat([BalanceSheet_2020, BalanceSheet_2021, BalanceSheet_2022])
BalanceSheet = BalanceSheet.drop("Unnamed: 0", axis=1)
# 补全股票代码
BalanceSheet['股票代码'] = BalanceSheet['股票代码'].apply(lambda x: f'{"0"*(6-len(x))}{x}' if len(x)<6 else x)

ProfitLost_2020 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股利润表2020.xlsx",
                               dtype={'股票代码': str})
ProfitLost_2020['所属年份'] = '2020'
ProfitLost_2021 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股利润表2021.xlsx",
                               dtype={'股票代码': str})
ProfitLost_2021['所属年份'] = '2021'
ProfitLost_2022 = pd.read_excel("./Playground/00_DataBase/02_Stock/A股利润表2022.xlsx",
                               dtype={'股票代码': str})
ProfitLost_2022['所属年份'] = '2022'
ProfitLost = pd.concat([ProfitLost_2020, ProfitLost_2021, ProfitLost_2022])
ProfitLost = ProfitLost.drop("Unnamed: 0", axis=1)
# 补全股票代码
ProfitLost['股票代码'] = ProfitLost['股票代码'].apply(lambda x: f'{"0"*(6-len(x))}{x}' if len(x)<6 else x)

# BalanceSheet.info(), ProfitLost.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15354 entries, 0 to 5092
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   序号        15354 non-null  int64         
 1   股票代码      15354 non-null  object        
 2   股票简称      15354 non-null  object        
 3   资产-货币资金   15228 non-null  float64       
 4   资产-应收账款   15146 non-null  float64       
 5   资产-存货     14848 non-null  float64       
 6   资产-总资产    15354 non-null  float64       
 7   资产-总资产同比  15354 non-null  float64       
 8   负债-应付账款   15190 non-null  float64       
 9   负债-预收账款   4685 non-null   float64       
 10  负债-总负债    15354 non-null  float64       
 11  负债-总负债同比  15354 non-null  float64       
 12  资产负债率     15354 non-null  float64       
 13  股东权益合计    15354 non-null  float64       
 14  公告日期      15354 non-null  datetime64[ns]
 15  所属年份      15354 non-null  object        
dtypes: datetime64[ns](1), float64(11), int64(1), object(3)
memo

(None, None)

In [171]:
# ROA = 税后净利润 / 总资产
def getROA(corp_code, year, BalanceSheet = BalanceSheet, ProfitLost = ProfitLost):
    asset = BalanceSheet[(BalanceSheet['股票代码'] == str(corp_code)) & (BalanceSheet['所属年份'] == str(year))]['资产-总资产']
    net_profit = ProfitLost[(ProfitLost['股票代码'] == str(corp_code)) & (ProfitLost['所属年份'] == str(year))]['净利润']
    if len(asset.tolist()) != 0 and len(net_profit.tolist()) != 0:
        return float(net_profit.values / asset.values)
    else:
#         print(f"corp {corp_code} has no info in ROA.")
        return None

# ROE = 税后净利润 / 净资产
def getROE(corp_code, year, BalanceSheet = BalanceSheet, ProfitLost = ProfitLost):
    equity = BalanceSheet[(BalanceSheet['股票代码'] == str(corp_code)) & (BalanceSheet['所属年份'] == str(year))]['股东权益合计']
    net_profit = ProfitLost[(ProfitLost['股票代码'] == str(corp_code)) & (ProfitLost['所属年份'] == str(year))]['净利润']
    if len(equity.tolist()) != 0 and len(net_profit.tolist()) != 0:
        return float(net_profit.values / equity.values)
    else:
#         print(f"corp {corp_code} has no info in ROE.")
        return None

# 成立时长(至2022年) ?2020年的企业数据反映的成立时长应该是与21年、22年的不同吧，每年增加1年?
def OperateDuration(corp_code, year):
    foundation_time = ak.stock_profile_cninfo(symbol=str(corp_code))['成立日期'].values.tolist()
    if len(foundation_time) != 0:
        foundation_year = foundation_time[0].split('-')[0]
        return int(year) - int(foundation_year) + 1
    else:
#         print(f"Corp {corp_code} has no info on founding date, set operating duration to \'None\'.")
        return None
    
# 总资产
def getAsset(corp_code, year, BalanceSheet = BalanceSheet):
    asset = BalanceSheet[(BalanceSheet['股票代码'] == str(corp_code)) & (BalanceSheet['所属年份'] == str(year))]['资产-总资产']
    return asset

In [ ]:
def getTobinQ(corp_code, year, )

In [174]:
df_PDF_main['ROA'] = df_PDF_main.apply(lambda row: getROA(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)
df_PDF_main['ROE'] = df_PDF_main.apply(lambda row: getROE(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_51824/2381933266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PDF_main['ROA'] = df_PDF_main.apply(lambda row: getROA(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)
/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_51824/2381933266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PDF_main['ROE'] = df_PDF_main.apply(lambda row: getROE(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)


In [175]:
df_PDF_main['OperateDuration'] = df_PDF_main.apply(lambda row: OperateDuration(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_51824/2035372471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PDF_main['OperateDuration'] = df_PDF_main.apply(lambda row: OperateDuration(corp_code = row['Corp_code'], year = row['Pub_year']), axis=1)


In [176]:
df_PDF_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PDF_name              1046 non-null   object 
 1   Corp_code             1046 non-null   object 
 2   Corp_name             1046 non-null   object 
 3   Corp_market           1046 non-null   object 
 4   Pub_year              1046 non-null   int64  
 5   Pub_type              1046 non-null   object 
 6   PDF_pages             1046 non-null   int64  
 7   img_num               1046 non-null   int64  
 8   img_size_avg          1046 non-null   float64
 9   img_num_pages_ratio   1046 non-null   float64
 10  img_size_pages_ratio  1046 non-null   float64
 11  dist_diag_ratio       961 non-null    float64
 12  Corp_industry_code    1040 non-null   object 
 13  Mono_idst             1046 non-null   bool   
 14  Poll_idst             1046 non-null   bool   
 15  ROA                  

In [177]:
df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main3.xlsx")

# ESG数据

In [9]:
BloombergESG = pd.read_excel("./Playground/00_DataBase/03_彭博ESG2006-2022（评级体系更新）/BloombergESG（匹配版）.xls",
                            dtype={'Stkcd': str})
BloombergESG['Stkcd'] = BloombergESG['Stkcd'].apply(lambda x: f'{"0"*(6-len(x))}{x}' if len(x)<6 else x)

In [14]:
BloombergESG = BloombergESG.rename(columns={'Stkcd':'Corp_code', 'Year':'Pub_year'})

In [16]:
df_PDF_main['Corp_code'] = df_PDF_main['Corp_code'].astype(str)
BloombergESG['Corp_code'] = BloombergESG['Corp_code'].astype(str)
df_PDF_main['Pub_year'] = df_PDF_main['Pub_year'].astype(str)
BloombergESG['Pub_year'] = BloombergESG['Pub_year'].astype(str)

In [19]:
df_PDF_main = pd.merge(df_PDF_main, BloombergESG, how='left',
        on=['Corp_code', 'Pub_year']).drop("Unnamed: 0", axis=1)

In [22]:
df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main4.xlsx")

# 专利信息（创新）

In [4]:
df_patent = pd.read_excel("./Playground/00_DataBase/04_国内外专利申请获得情况表204112695 (1)/PT_LCDOMFORAPPLY.xlsx")
df_patent.info()

/Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48011 entries, 0 to 48010
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Symbol         48011 non-null  object
 1   EndDate        48011 non-null  object
 2   StateTypeCode  48011 non-null  object
 3   Area           48011 non-null  object
 4   ApplyTypeCode  48011 non-null  object
 5   ApplyType      48011 non-null  object
 6   Patents        38033 non-null  object
 7   Invention      34650 non-null  object
dtypes: object(8)
memory usage: 2.9+ MB


In [7]:
df_patent['EndDate'].unique()

array(['统计截止日期', '没有单位', '2021-12-31', '2016-12-31', '2017-12-31',
       '2018-12-31', '2019-12-31', '2020-12-31', '2022-12-31',
       '2008-12-31', '2013-12-31', '2014-12-31', '2015-12-31',
       '2011-12-31', '2007-12-31', '2012-12-31', '2009-12-31',
       '2010-12-31'], dtype=object)

In [17]:
df_patent = df_patent.rename(columns={'Symbol':'Corp_code'})
df_patent

,Corp_code,EndDate,StateTypeCode,Area,ApplyTypeCode,ApplyType,Patents,Invention
0,证券代码,统计截止日期,报表类型,地区,申请类型编码,申请类型,专利,发明专利
1,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,件,件
2,000004,2021-12-31,1,1,S5204,截至报告期末累计获得,NaN,14
3,000008,2016-12-31,1,1,S5204,截至报告期末累计获得,NaN,56
4,000008,2017-12-31,1,1,S5201,已申请,123,NaN
...,...,...,...,...,...,...,...,...
48006,900956,2015-12-31,1,1,S5203,已授权,56,NaN
48007,900956,2017-12-31,1,1,S5201,已申请,12,4
48008,900956,2018-12-31,1,1,S5201,已申请,114,NaN
48009,900956,2018-12-31,1,1,S5203,已授权,63,13


In [14]:
df_PDF_main['EndDate'] = df_PDF_main['Pub_year'].apply(lambda x: '-'.join([str(x), '12', '31']))

In [101]:
df_patent_gain = df_patent[(df_patent['ApplyTypeCode'] == 'S5204') & (df_patent['Area']==1) & (df_patent['StateTypeCode']==1)]

In [104]:
df_PDF_main['Corp_code'] = df_PDF_main['Corp_code'].astype(str)
df_patent_gain['Corp_code'] = df_patent_gain['Corp_code'].astype(str)

df_PDF_main = df_PDF_main.merge(df_patent_gain[df_patent_gain['ApplyTypeCode']=="S5204"][['Corp_code', 'EndDate', 'Patents', 'Invention']],
                                 how='left', on=['Corp_code', 'EndDate'])

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_7667/2785880780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_patent_gain['Corp_code'] = df_patent_gain['Corp_code'].astype(str)


In [106]:
df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main5.xlsx",
                    index=False)

# 分析师研报关注度

In [108]:
df_attention = pd.read_excel("./Playground/00_DataBase/05_上市公司基本信息特色指标表132631814/AF_CFEATUREPROFILE.xlsx")
df_attention.info()

/Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55661 entries, 0 to 55660
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Stknmec          55661 non-null  object
 1   Stkcd            55661 non-null  object
 2   Accper           55661 non-null  object
 3   CompanySize      55355 non-null  object
 4   Big4             55360 non-null  object
 5   Outside          55360 non-null  object
 6   AnaAttention     34427 non-null  object
 7   ReportAttention  34841 non-null  object
 8   CompanyOpacity   37907 non-null  object
 9   RegisterCapital  55661 non-null  object
dtypes: object(10)
memory usage: 4.2+ MB


In [112]:
df_attention = df_attention.rename(columns={'Stkcd':'Corp_code', 'Accper':'EndDate'})

In [115]:
df_PDF_main = df_PDF_main.merge(df_attention[['Corp_code', 'EndDate', 'AnaAttention','ReportAttention']], 
                  how='left', on=['Corp_code', 'EndDate'])

In [117]:
df_PDF_main.to_excel("./Playground/04_DataIntegration/df_PDF_main6.xlsx",
                    index=False)